In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from pathlib import Path
import csv

In [ ]:
options = Options()
options.headless = True

tourn_id = 30030
largest_round_id = 1171709
url = "https://www.tabroom.com/index/tourn/results/index.mhtml?tourn_id={tourn_id}"
url = url.replace("{tourn_id}", str(tourn_id))
tourn_name = "NSDA Nationals Qualifier"
EVENT_IDENTIFIER = "Event Results"
OO_IDENTIFIER = "OO Results"
DROPDOWN_OPTION = "Original Oratory"
OUTPUT_CSV_DIR = Path().resolve().parent / "output_csv"
OUTPUT_CSV_SUBDIR = OUTPUT_CSV_DIR / tourn_name
OUTPUT_CSV_SUBDIR.mkdir(parents=True, exist_ok=True)

In [ ]:
def navigate_to_oo_page(driver) -> None:
    driver.get(url)
    select2_container = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".select2-selection--single"))
    )
    select2_container.click()
    option_xpath = f"//li[contains(@class, 'select2-results__option') and . = '{DROPDOWN_OPTION}']"
    desired_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, option_xpath))
    )
    
    desired_option.click()

In [ ]:
driver = webdriver.Firefox(options=options)
navigate_to_oo_page(driver)
sidenotes = driver.find_elements(By.CLASS_NAME, "sidenote")
matched_element = None
found = False

for sidenote in sidenotes:
    if OO_IDENTIFIER in sidenote.text:
        assert found is not True
        matched_element = sidenote
        found = True
a_tags = matched_element.find_elements(By.TAG_NAME, "a")
[a_tag.text for a_tag in a_tags]
links_hrefs = [[link.get_attribute('href'), link.text.strip()] for link in a_tags]

In [ ]:
for link, round_name in links_hrefs:
    driver.get(link)
    output_csv_file = (OUTPUT_CSV_SUBDIR / round_name.replace(" ", "_")).with_suffix(".csv")  # TODO: lstrip & rstrip?
    table = driver.find_element(By.TAG_NAME, "table")
    first_row = table.find_element(By.TAG_NAME, "tr")
    headers = first_row.find_elements(By.TAG_NAME, "th")
    header_row = ["round_stage"] + [header.text for header in headers if header.text != ""]
    with open(output_csv_file, 'w+', newline='', encoding='utf-8') as file:
        # TODO: error handling, e.g. len(s) == 1 and len(header_row) in s
        writer = csv.writer(file)
        writer.writerow(header_row)
        rows = driver.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")

            if len(columns) == 0:
                continue
            
            column_texts = [col.text for col in columns]
            writer.writerow([round_name] + column_texts)
driver.quit()